In [146]:
import collections
from langdetect import detect
from nltk import pos_tag
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
from nltk.stem import wordnet, WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import pyLDAvis.sklearn
import re
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import unicodedata
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:110% !important; }</style>"))

### Create Top Pizza Data Set

In [ ]:
# # read in yelp reviews and business data
# reviews = pd.read_csv('../Assets/yelp_academic_dataset_review.csv')
# biz = pd.read_csv('../Assets/yelp_academic_dataset_business.csv')

# pizza_biz = ['Pizza Hut', 'Domino\'s Pizza', 'Little Caesars', 
#              'Papa John\'s Pizza','Papa Murphy\'s', 'Grimaldi\'s Pizzeria',
#              'Mellow Mushroom', 'Blaze Fast-Fire\'d Pizza']

# # filter businesses to only include top pizza delivery
# top_pizza_biz = biz.loc[biz['name'].isin(pizza_biz), :]
                        
# # merge reviews to the LV_Pizza businesses
# top_pizza_reviews = (top_pizza_biz.loc[:, ['business_id', 'name', 'stars', 'state', 
#                                            'city', 'neighborhood','address', 'postal_code']]
#                                   .rename(columns={'stars': 'biz_stars'})
#                                   .merge(reviews, how='inner', on='business_id'))
# # define restaurant type
# restaurant_type = pd.DataFrame({'name': pizza_biz,
#                                 'type': ['Delivery', 'Delivery', 'Delivery', 'Delivery', 
#                                          'Take and Bake', 'Dine-in', 'Dine-in', 'Fast Casual']})
# # merge restaurant type to dataframe
# top_pizza_reviews = top_pizza_reviews.merge(restaurant_type, how='inner', on='name')

# # Write to csv
# top_pizza_reviews.to_csv('../Assets/top_pizza_reviews.csv', index=False)   

### Create the Las Vegas Pizza dataset and write to CSV

In [ ]:
# # read in yelp reviews and business data
# reviews = pd.read_csv('../Assets/yelp_academic_dataset_review.csv')
# biz = pd.read_csv('../Assets/yelp_academic_dataset_business.csv')

# # filter businesses to only include places in Las Vegas that have "pizza" in their category
# lv_pizza_biz = biz.loc[(biz['city']=="Las Vegas") & 
#                        (biz['state']=='NV') &
#                        (biz['categories'].str.lower().str.contains('pizza')), :]

# # merge reviews to the LV_Pizza businesses
# lv_pizza_reviews = (lv_pizza_biz.loc[:, ['business_id', 'name', 'stars', 'state', 
#                                          'city', 'neighborhood','address', 'postal_code']]
#                                 .rename(columns={'stars': 'biz_stars'})
#                                 .merge(reviews, how='inner', on='business_id'))

# # Write to csv
# lv_pizza_reviews.to_csv('../Assets/lv_pizza_reviews.csv', index=False)


# # read in tips
# tips = pd.read_csv('../Assets/yelp_academic_dataset_tip.csv')

# # get tips for lv_pizza businesses
# biz_ids = lv_pizza_reviews.loc[:, ['business_id', 'name', 'stars', 'state', 
#                                   'city', 'neighborhood','address', 'postal_code']].drop_duplicates()
# lv_pizza_tips = biz_ids.merge(tips, how='inner', on='business_id')

# # write to csv
# lv_pizza_tips.to_csv('../Assets/lv_pizza_tips.csv', index=False)


### Prepare lv_pizza_reviews dataset

In [ ]:
# # read in lv_pizza_reviews and assign proper data types to columns
# df = (pd.read_csv('../Assets/lv_pizza_reviews.csv')
#         .assign(date=lambda x: pd.to_datetime(x['date'], errors='coerce')))
# print(df.info())

# tips = pd.read_csv('../Assets/lv_pizza_tips.csv')
# print(tips.info())

# low_rating = df.loc[df['stars'] <= 2, :]
# high_rating = df.loc[df['stars'] >= 4, :]


### Load top pizza reviews dataset

In [3]:
# read in top_pizza reviews 
df = (pd.read_csv('../Assets/top_pizza_reviews.csv')
        .assign(date=lambda x: pd.to_datetime(x['date'], errors='coerce')))
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20877 entries, 0 to 20876
Data columns (total 17 columns):
business_id     20877 non-null object
name            20877 non-null object
biz_stars       20877 non-null float64
state           20877 non-null object
city            20877 non-null object
neighborhood    8698 non-null object
address         20877 non-null object
postal_code     20877 non-null object
review_id       20877 non-null object
text            20877 non-null object
date            20877 non-null datetime64[ns]
cool            20877 non-null int64
funny           20877 non-null int64
useful          20877 non-null int64
stars           20877 non-null int64
user_id         20877 non-null object
type            20877 non-null object
dtypes: datetime64[ns](1), float64(1), int64(4), object(11)
memory usage: 2.7+ MB
None


### Remove Non-English Reviews

In [19]:
def detectexception(text):
    try:
        return detect(text)
    except:
        return 'NaL' # return not a language
    
# detect language
df['lan']=df['text'].apply(lambda x:detectexception(x))

In [21]:
df = df.loc[df['lan']=='en', :]

### EDA

In [66]:
rating_by_biz = (df.groupby(['name'], as_index=False)
                   .agg({'stars': 'mean',
                         'business_id': 'count'})
                   .rename(columns={'name': 'Restaurant',                    
                                    'stars': 'Avg Rating',
                                    'business_id': 'Number_Reviews'})
                   .sort_values(['Avg Rating'], ascending=False))

rating_by_type = (df.groupby(['type'], as_index=False)
                    .agg({'stars': 'mean',
                          'business_id': 'count'})
                    .rename(columns={'stars': 'Avg Rating',
                                     'business_id': 'Number_Reviews',
                                     'type': 'Restaurant Type'})
                    .sort_values(['Avg Rating'], ascending=False))


rating_count_by_biz = (df.pivot_table(index='name', 
                                      columns='stars',
                                      values='review_id',
                                      aggfunc='count')
                         .reset_index()
                         .assign(count=lambda x: x[1] + x[2] + x[3] + x[4] + x[5])
                         .assign(pct_1=lambda x: x[1]/x['count'],
                                 pct_2=lambda x: x[2]/x['count'],
                                 pct_3=lambda x: x[3]/x['count'],
                                 pct_4=lambda x: x[4]/x['count'],
                                 pct_5=lambda x: x[5]/x['count']))

print(rating_by_biz)
print()
print(rating_by_type)
print()
print(rating_count_by_biz)


                Restaurant  Avg Rating  Number_Reviews
0  Blaze Fast-Fire'd Pizza    4.203528            2211
2      Grimaldi's Pizzeria    3.988988            5176
6            Papa Murphy's    3.943775             996
4          Mellow Mushroom    3.321091            2750
3           Little Caesars    2.843220             236
1           Domino's Pizza    2.593168            3220
5        Papa John's Pizza    2.371194            2069
7                Pizza Hut    2.134164            4174

  Restaurant Type  Avg Rating  Number_Reviews
2     Fast Casual    4.203528            2211
3   Take and Bake    3.943775             996
1         Dine-in    3.757255            7926
0        Delivery    2.354366            9699

stars                     name     1    2    3     4     5  count     pct_1  \
0      Blaze Fast-Fire'd Pizza   130  117  157   576  1231   2211  0.058797   
1               Domino's Pizza  1473  297  277   413   760   3220  0.457453   
2          Grimaldi's Pizzeria   376

### Predict Sentiment

In [23]:
def normalize_accented_characters(text):
    '''
    removes accents from characters and replaces with non-accented equivalent
    '''
    if type(text) != str:
        text = (unicodedata.normalize('NFKD', text.decode('utf-8'))
                           .encode('ascii', 'ignore'))
    return text

In [77]:
def analyze_sentiment_sentiwordnet_lexicon(review, value='sentiment', verbose=False):
    '''
    analyzes sentiment of text using 
    '''
    # pre-process text
    review = normalize_accented_characters(review)
    
    # tokenize, lematize, and POS tag text tokens
    wordnet_lemmatizer = WordNetLemmatizer()
    text_tokens = word_tokenize(review)
    lemma_tokens = [wordnet_lemmatizer.lemmatize(token) for token in text_tokens]
    tagged_text = pos_tag(lemma_tokens)
    pos_score = neg_score = token_count = obj_score = 0
    
    # get wordnet synsets based on POS tags for NN, VB, JJ, RB
    # get sentiment scores if synsets are found
    for word, tag in tagged_text:  
        ss_set = None
        if 'NN' in tag and list(swn.senti_synsets(word, 'n')): # singular noun
            ss_set = list(swn.senti_synsets(word, 'n'))[0]
        elif 'VB' in tag and list(swn.senti_synsets(word, 'v')): # Verb
            ss_set = list(swn.senti_synsets(word, 'v'))[0]
        elif 'JJ' in tag and list(swn.senti_synsets(word, 'a')): # Adjective
            ss_set = list(swn.senti_synsets(word, 'a'))[0]
        elif 'RB' in tag and list(swn.senti_synsets(word, 'r')): # adverb
            ss_set = list(swn.senti_synsets(word, 'r'))[0]
        
        # if senti-synset is found
        if ss_set:
            # add scores for all found synsets
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
        
        # if no words have synset, return None 
        if token_count == 0:
            return None
        
    # aggregate final scores
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = 'positive' if norm_final_score >= 0 else 'negative'
    if verbose:
        norm_obj_score = round(float(obj_score) / token_count, 2)
        norm_pos_score = round(float(pos_score) / token_count, 2)
        norm_neg_score = round(float(neg_score) / token_count, 2)
        # to display results in a nice table
        sentiment_frame = pd.DataFrame([[final_sentiment, norm_obj_score,
                                         norm_pos_score, norm_neg_score,
                                         norm_final_score]],
                                         columns=pd.MultiIndex(
                                             levels=[['SENTIMENT STATS:'],
                                                ['Predicted Sentiment',
                                                 'Objectivity',
                                                 'Positive', 'Negative',
                                                 'Overall']],
                                              labels=[[0,0,0,0,0],
                                                      [0,1,2,3,4]]))
    if value=='sentiment':
        return final_sentiment
    elif value=='score':
        return norm_final_score

In [78]:
# predict sentiment using synsets
df['pred_sentiment'] = df['text'].apply(lambda x: analyze_sentiment_sentiwordnet_lexicon(x))
df['pred_sent_score'] = df['text'].apply(lambda x: analyze_sentiment_sentiwordnet_lexicon(x, 'score')) 
df.loc[df['pred_sentiment'].isna(), 'pred_sentiment'] = 'neutral' # assign neutral to null values

In [79]:
sent_by_restaurant = (df.pivot_table(index='name', 
                                     columns='pred_sentiment', 
                                     values='review_id',
                                     aggfunc='count')
                        .reset_index()
                        .assign(count=lambda x: x['positive'] + x['negative'] + x['neutral'])
                        .assign(pct_pos=lambda x: x['positive']/x['count'],
                                pct_neg=lambda x: x['negative']/x['count'],
                                pct_neutral=lambda x: x['neutral']/x['count']))

sent_by_star = (df.pivot_table(index='stars', 
                               columns='pred_sentiment', 
                               values='review_id',
                               aggfunc='count')
                  .reset_index()
                  .assign(count=lambda x: x['positive'] + x['negative'] + x['neutral'])
                  .assign(pct_pos=lambda x: x['positive']/x['count'],
                          pct_neg=lambda x: x['negative']/x['count'],
                          pct_neutral=lambda x: x['neutral']/x['count']))
            
sent_by_star


pred_sentiment,stars,negative,neutral,positive,count,pct_pos,pct_neg,pct_neutral
0,1,1362,3553,1123,6038,0.185989,0.225571,0.588440
1,2,222,1199,477,1898,0.251317,0.116965,0.631718
2,3,142,1412,620,2174,0.285189,0.065317,0.649494
3,4,151,2556,1422,4129,0.344393,0.036571,0.619036
4,5,146,4064,2383,6593,0.361444,0.022145,0.616411


### Create preprocessing tools

In [148]:
# create custom list of stopwords
stopwords = set(stopwords.words('english')).union(set(ENGLISH_STOP_WORDS)).union(set(['pizza']))

In [149]:
# create custome tokenizer and stemmer

def tokenize_and_lemmatize(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in sent_tokenize(text) for word in word_tokenize(sent)]
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    filtered_tokens = []
    for token in lemma_tokens:
        if re.search(r'[a-zA-Z\-][a-zA-Z\-]{3,}', token):  # character only words of length >= 3
            filtered_tokens.append(token)
    return filtered_tokens

### clustering function

In [141]:
def kmeans(data, num_clusters):
    # converts collection of text documents (reviews) to a matrix of token counts
    vectorizer = CountVectorizer(min_df=5,     # word must appear at least 5 times to be included
                                 max_df=0.9,   # don't include words that are found in 90% of documents
                                 stop_words=stopwords,       # remove stop words
                                 lowercase=True,             # lowercase all words
                                 strip_accents='unicode',    # remove accents
#                                  token_pattern=r'[a-zA-Z\-][a-zA-Z\-]{3,}', # character only words of length >= 3
                                 tokenizer=tokenize_and_lemmatize,    # uses tokenize_and_stem function to tokenize
                                 ngram_range=(1,1))          # consider single words and pairs of words
    # learn the vocabulary dictionary and return the term-document matrix
    count_matrix = vectorizer.fit_transform(data['text'])  

    # get feature names
    feature_names = vectorizer.get_feature_names()
    
    tfidf_transformer = TfidfTransformer()  # instatiate the object
    tfidf_matrix = tfidf_transformer.fit_transform(count_matrix)  # transform count matrix to tf-idf matrix
    
    # run kmeans
    km = KMeans(n_clusters=num_clusters, n_jobs=-1) # instantiate kmeans object, with n clusters, and tell it to use all processors
    km.fit(tfidf_matrix)
    clusters = km.labels_.tolist()
    data.loc[:,'cluster'] = clusters  # assign cluster results to dataframe
    
    # get details of clusters
    cluster_details = {}

    # get cluster centroids
    ordered_centroids = km.cluster_centers_.argsort()[:, ::-1]

    # get key features for each cluster
    # and also list restaurants and reviews associated with that cluster
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}  # assign cluster number as dictionary
        cluster_details[cluster_num]['cluster_num'] = cluster_num  # assign cluster num
        key_features = [feature_names[index] for index in ordered_centroids[cluster_num, :20]] # top 20 features of cluster
        cluster_details[cluster_num]['key_features'] = key_features

        businesses = data.loc[data['cluster'] == cluster_num, 'name'].unique().tolist()  # list of biz in cluster
        review_ids = data.loc[data['cluster'] == cluster_num, 'review_id'].unique().tolist() # list of reviews in cluster
        stars = data.loc[data['cluster'] == cluster_num, 'stars'].unique().tolist() # 
        cluster_details[cluster_num]['businesses'] = businesses  # assign businesses
        cluster_details[cluster_num]['review_ids'] = review_ids  # assign review_ids
    cluster_df = pd.DataFrame(cluster_details).T
    
    return data, cluster_df

### Clusters for Low Sentiment and Rating

In [154]:
low_df, low_df_clusters = kmeans(data.loc[(data['stars'] <= 2) 
                                          & (data['pred_sentiment'] == 'negative'), :], 3)
for i in range(low_df_clusters.index.size):
    print('{}\n{}\n\n'.format(i, low_df_clusters.loc[i, 'key_features']))

0
['cheese', 'crust', 'sauce', 'wing', 'ordered', 'like', 'topping', 'extra', 'time', 'order', 'chicken', 'pepperoni', 'good', 'worst', 'taste', 'cold', 'location', 'domino', 'large', 'place']


1
['order', 'minute', 'hour', 'called', 'delivery', 'time', 'said', 'told', 'ordered', 'manager', 'cold', 'phone', 'food', 'driver', 'online', 'store', 'service', 'location', 'waited', 'wrong']


2
['service', 'customer', 'time', 'food', 'horrible', 'place', 'order', 'rude', 'location', 'worst', 'terrible', 'manager', 'staff', 'ordered', 'like', 'delivery', 'table', 'called', 'make', 'experience']




/Users/mwitebsky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Clusters for high sentiment and rating

In [156]:
high_df, high_df_clusters = kmeans(data.loc[(data['stars'] >= 4)
                                            & (data['pred_sentiment'] == 'positive'), :], 3)
for i in range(high_df_clusters.index.size):
    print('{}\n{}\n\n'.format(i, high_df_clusters.loc[i, 'key_features']))

0
['order', 'delivery', 'domino', 'online', 'time', 'ordered', 'good', 'fast', 'location', 'friendly', 'john', 'papa', 'ordering', 'great', 'service', 'minute', 'driver', 'love', 'like', 'food']


1
['great', 'service', 'food', 'place', 'friendly', 'staff', 'good', 'love', 'beer', 'time', 'customer', 'best', 'amazing', 'price', 'awesome', 'happy', 'delicious', 'nice', 'definitely', 'atmosphere']


2
['good', 'crust', 'place', 'topping', 'love', 'like', 'sauce', 'best', 'time', 'salad', 'really', 'cheese', 'fresh', 'great', 'delicious', 'nice', 'pepperoni', 'make', 'service', 'grimaldi']




/Users/mwitebsky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Clusters for discordant sentiment and rating

In [157]:
discordant_df, discordant_df_clusters = kmeans(data.loc[((data['stars'] >= 4) & (data['pred_sentiment'] == 'negative'))
                                                        | ((data['stars'] <= 2) & (data['pred_sentiment'] == 'positive')), :], 3)
for i in range(discordant_df_clusters.index.size):
    print('{}\n{}\n\n'.format(i, discordant_df_clusters.loc[i, 'key_features']))

0
['sauce', 'cheese', 'crust', 'topping', 'ordered', 'like', 'time', 'good', 'wing', 'extra', 'taste', 'better', 'order', 'pepperoni', 'chicken', 'domino', 'papa', 'location', 'place', 'half']


1
['order', 'minute', 'called', 'delivery', 'time', 'said', 'hour', 'told', 'store', 'ordered', 'online', 'phone', 'manager', 'driver', 'service', 'location', 'asked', 'customer', 'later', 'delivered']


2
['service', 'great', 'food', 'place', 'good', 'time', 'customer', 'location', 'like', 'order', 'staff', 'table', 'really', 'better', 'beer', 'wait', 'minute', 'server', 'people', 'came']




/Users/mwitebsky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Run LDA

In [143]:
def lda(data, num_topics):
    # converts collection of text documents (reviews) to a matrix of token counts
    vectorizer = CountVectorizer(min_df=5,     # word must appear at least 5 times to be included
                                 max_df=0.9,   # don't include words that are found in 90% of documents
                                 stop_words=stopwords,       # remove stop words
                                 lowercase=True,             # lowercase all words
                                 strip_accents='unicode',    # remove accents
#                                  token_pattern=r'[a-zA-Z\-][a-zA-Z\-]{3,}', # character only words of length >= 3
                                 tokenizer=tokenize_and_lemmatize,    # uses tokenize_and_stem function to tokenize
                                 ngram_range=(1,1))          # consider single words and pairs of words
    # learn the vocabulary dictionary and return the term-document matrix
    count_matrix = vectorizer.fit_transform(data['text'])  


    # Build a Latent Dirichlet Allocation Model
    lda_model = LatentDirichletAllocation(n_components=num_topics,  # specify number of topics to create
                                          max_iter=10, 
                                          learning_method='online')  # in general, online is faster method

    lda_out = lda_model.fit_transform(count_matrix)
    
    # create cool viz of LDA topics
    pyLDAvis.enable_notebook()
    panel = pyLDAvis.sklearn.prepare(lda_model, count_matrix, vectorizer, mds='tsne')
    return panel

In [150]:
lda_viz = lda(df, 5)
lda_viz

/Users/mwitebsky/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      -24.045179   30.077118       1        1  35.320085
4      -43.604946 -127.125870       2        1  23.170073
1      231.319733  -34.602901       3        1  16.930530
3      109.520325 -127.410263       4        1  15.503091
2      120.668678   78.760376       5        1   9.076221, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
3640  Default  11776.000000       order  11776.000000  30.0000  30.0000
2399  Default   6934.000000       great   6934.000000  29.0000  29.0000
4576  Default   3749.000000        said   3749.000000  28.0000  28.0000
936   Default   4234.000000      cheese   4234.000000  27.0000  27.0000
4618  Default   4168.000000       sauce   4168.000000  26.0000  26.0000
3364  Default   5054.000000      minute   5054.000000  25.0000  25.0000
3215  Default   3273.000000     manager   3273.000000  24.0000  24.0000
769   Default   2918.000000      called   2918.000000  23.0000  23.0000
3444  Default   2598.000000    mushroom   2598.000000  22.0000  22.0000
1356  Default   5190.000000       crust   5190.000000  21.0000  21.0000
1489  Default   3702.000000    delivery   3702.000000  20.0000  20.0000
5508  Default   2687.000000        told   2687.000000  19.0000  19.0000
2356  Default   9273.000000        good   9273.000000  18.0000  18.0000
3132  Default   5605.000000    location   5605.000000  17.0000  17.0000
5479  Default  10659.000000        time  10659.000000  16.0000  16.0000
340   Default   2352.000000       asked   2352.000000  15.0000  15.0000
4726  Default   7690.000000     service   7690.000000  14.0000  14.0000
3817  Default   1914.000000   pepperoni   1914.000000  13.0000  13.0000
3745  Default   2280.000000        papa   2280.000000  12.0000  12.0000
1664  Default   2245.000000      domino   2245.000000  11.0000  11.0000
4578  Default   3066.000000       salad   3066.000000  10.0000  10.0000
3617  Default   2093.000000      online   2093.000000   9.0000   9.0000
1382  Default   3529.000000    customer   3529.000000   8.0000   8.0000
3644  Default   6408.000000     ordered   6408.000000   7.0000   7.0000
3163  Default   3337.000000        love   3337.000000   6.0000   6.0000
2249  Default   2429.000000    friendly   2429.000000   5.0000   5.0000
3861  Default   1814.000000       phone   1814.000000   4.0000   4.0000
5529  Default   4373.000000     topping   4373.000000   3.0000   3.0000
2898  Default   1683.000000        john   1683.000000   2.0000   2.0000
2644  Default   3173.000000        hour   3173.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
1900   Topic5    332.233791     evening    335.680861   2.3892  -5.2650
2139   Topic5    154.056385       floor    161.759231   2.3507  -6.0336
1623   Topic5    199.762228  disgusting    214.823838   2.3268  -5.7738
651    Topic5    260.533507  breadstick    319.508275   2.1955  -5.5082
2459   Topic5    194.950935        hair    233.958209   2.2171  -5.7981
4084   Topic5    224.523393     product    294.154843   2.1294  -5.6569
2471   Topic5    453.155557        hand    754.334242   1.8899  -4.9547
648    Topic5    485.521335       bread    888.578557   1.7951  -4.8857
411    Topic5    551.482397        away   1237.802714   1.5910  -4.7583
3145   Topic5    467.320441        look   1059.097743   1.5814  -4.9239
1682   Topic5    476.552341       dough   1165.780904   1.5049  -4.9043
3405   Topic5    408.200828       money    930.940986   1.5751  -5.0591
3132   Topic5   1132.413037    location   5605.170653   0.8002  -4.0388
936    Topic5    938.840224      cheese   4234.459126   0.8931  -4.2262
3208   Topic5    345.440335      making    721.387670   1.6632  -5.2261
1982   Topic5    459.225150       extra   1519.763079   1.2027  -4.9413
3089   Topic5    892.80

### Visualize Clusters  - DON'T RUN.  DOESNT WORK!

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.manifold import MDS
# from sklearn.metrics.pairwise import cosine_similarity
# import random
# from matplotlib.font_manager import FontProperties

    
# # generate random color for clusters
# def generate_random_color():
#     color = '#%06x' % random.randint(0, 0xFFFFFF)
#     return color
    
# plot_size=(16,8)

# # define markers for clusters
# markers = ['o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd']

# # build cosine distance matrix
# cosine_distance = 1 - cosine_similarity(tfidf_matrix)

# # dimensionality reduction using MDS
# mds = MDS(n_components=2, dissimilarity="precomputed",
#           random_state=1)

# # get coordinates of clusters in new low-dimensional space
# plot_positions = mds.fit_transform(cosine_distance)
# x_pos, y_pos = plot_positions[:, 0], plot_positions[:, 1]

# # build cluster plotting data
# cluster_color_map = {}
# cluster_name_map = {}

# for cluster_num, cluster_details in cluster_details.items():
#     # assign cluster features to unique label
#     cluster_color_map[cluster_num] = generate_random_color()
#     cluster_name_map[cluster_num] = ', '.join(cluster_details['key_features'][:5]).strip()

# # map each unique cluster label with its coordinates and movies
# cluster_plot_frame = pd.DataFrame({'x': x_pos,
#                                    'y': y_pos,
#                                    'label': data['cluster'].values.tolist(),
#                                    'stars': data['stars'].values.tolist()})
# grouped_plot_frame = cluster_plot_frame.groupby('label')

# # set plot figure size and axes
# fig, ax = plt.subplots(figsize=plot_size)
# ax.margins(0.05)

# # plot each cluster using co-ordinates and movie titles
# for cluster_num, cluster_frame in grouped_plot_frame:
#     marker = markers[cluster_num] if cluster_num < len(markers) \
#              else np.random.choice(markers, size=1)[0]
# ax.plot(cluster_frame['x'], cluster_frame['y'],
#         marker=marker, linestyle='', ms=12,
#         label=cluster_name_map[cluster_num],
#         color=cluster_color_map[cluster_num], mec='none')
# ax.set_aspect('auto')
# ax.tick_params(axis= 'x', which='both', bottom='off', top='off',
#                labelbottom='off')
# ax.tick_params(axis= 'y', which='both', left='off', top='off',
#                labelleft='off')
# fontP = FontProperties()
# fontP.set_size('small')
# ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01),
#           fancybox=True,
#           shadow=True, ncol=5, numpoints=1, prop=fontP)

# #add labels as the film titles
# for index in range(len(cluster_plot_frame)):
#     ax.text(cluster_plot_frame.ix[index]['x'],
#     cluster_plot_frame.ix[index]['y'],
#     cluster_plot_frame.ix[index]['stars'], size=8)

# # show the plot
# plt.show()

### view highest score tfidf grams (Didn't Use)

In [ ]:
# # changes count matrix to dense (can see gram mentions per review)
# grams = pd.DataFrame(count_matrix.todense(), index=data.index, columns=feature_names)
# grams['review_id'] = data['review_id']

# # format to long matrix
# grams = (pd.melt(grams.reset_index(), 
#                  id_vars=['index','review_id'],
#                  value_name='gram_ct')
#             .query('gram_ct > 0')  # only include where gram was mentioned
#             .sort_values(['index','review_id']))

# # changes tfidf matrix to dense (can see gram score per review)
# tfidf = pd.DataFrame(tfidf_matrix.todense(), index=data.index, columns=feature_names)
# tfidf['review_id'] = data['review_id']

# # format to long matrix
# tfidf = (pd.melt(tfidf.reset_index(), 
#                  id_vars=['index','review_id'],
#                  value_name='tfidf')
#             .query('tfidf > 0'))  # only include where it has a tfidf score

In [ ]:
# gram_summary = (grams.merge(tfidf, 
#                             on=['index','review_id','variable'])
#                      .set_index('index'))

In [ ]:
# gram_summary.groupby(['variable'], as_index=False).agg({'tfidf': 'mean'}).sort_values(['tfidf'], ascending=False)